**The code below demonstrates how sensitive data detection works and how customers use data in their algorithms without sensitive columns**

Sample datasets are created

In [176]:
import pandas as pd  
  
data = {'column_01': ['0.928783635', '0.928783635', '0.928783635', '0.36374635', '0.345688899', '0.345683635', '0.928783635', '0.937583635', '0.743483635', '0.929483874', '0.241238947', '0.929834793'], 'column_02': ['99040217', '78499217', '67589076', '1983050127', '198305010', '67589076', '67589076', '67589076', '67589076', '19870217', '73850217', '19375517'], 'column_03': ['20101101-4555', '19630701-8990', '19930701-8277', '19930701-8277', '19930701-8277', '19930701-8277', '19830501-7274', '19830501-7274', '198305017274', '19830501-7274', '19791001-6571', '20010611-1264'], 'column_04': ['98339063', '63544063', '67589063', '67589076', '67589076', '67589076', '67589076', '67587377', '67587377', '67587377', '67587377', '67580933'], 'column_05': ['6677-8899-3465-5366', '4912-8899-3465-8499', '4912-8899-3465-5366', '4912-5674-3465-1234', '4912-5674-3465-1234', '4912-5674-3465-1234', '4912-5674-3465-1234', '4912567434651234', '4912567434651234', '4912-5674-3465-1234', '4912-7733-3465-9824', '4912-5674-7388-8395'], 'column_06': [7, 2, 4, 4, 5, 9, 1, 8, 9, 10, 0, 12], 'column_07': [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1]}  
data_01_df = pd.DataFrame(data)  

data = {'column_a': ['0.743483635', '0.929483874', '0.241238947', '0.929834793'], 'column_b': ['19830501-7274', '19830501', '67589076', '19870217'], 'column_c': ['19830501-7274', '19830501-7274', '198305017274', '19830501-7274'], 'column_d': ['67589076', '67589076', '67589076', '67589076'], 'column_e': ['4912-5674-3465-1234', '4912567434651234', '4912567434651234', '4912-5674-3465-1234'], 'column_f': ['a', 'a', 'b', 'a']}  
data_02_df = pd.DataFrame(data)  

data = {'c1': ['19830501-7274', '19830501', '67589076', '19870217'], 'c2': ['67589076', '67589076', '67589076', '67589076'], 'c3': ['4912-5674-3465-1234', '4912567434651234', '4912567434651234', '4912-5674-3465-1234'], 'c3': ['1', '1', '1', '2']}  
data_03_df = pd.DataFrame(data)  


data_01_df.to_csv("data_10001.csv", index=False)
data_02_df.to_csv("data_10002.csv", index=False)
data_03_df.to_csv("data_10003.csv", index=False)



---





A sample customer - dataset table is created, that shows which customer has access to which datasets

In [177]:
data = {'customer_id': ['101', '102', '103'], 'timestamp': ['2022.01.01', '2022.01.03', '2022.01.05']}  
customer_df = pd.DataFrame(data)  
customer_df.to_csv('customer_table.csv', index=False)

data = {'dataset_id': ['10001', '10002', '10003'], 'timestamp': ['2022.02.01', '2022.02.03', '2022.02.05']}  
dataset_df = pd.DataFrame(data)  
dataset_df.to_csv('dataset_table.csv', index=False)


data = {'customer_id': ['101', '101', '102', '103', '104'], 'dataset_id': ['10001', '10002', '10002', '10003', '10001'], 'timestamp': ['2022.01.01', '2022.01.02', '2022.03.12', '2022.03.15', '2022.06.07']}  
customer_dataset_df = pd.DataFrame(data)  
customer_dataset_df.to_csv('customer_dataset_table.csv', index=False)

customer_dataset_df

,customer_id,dataset_id,timestamp
0,101,10001,2022.01.01
1,101,10002,2022.01.02
2,102,10002,2022.03.12
3,103,10003,2022.03.15
4,104,10001,2022.06.07




---



Validation methods. At this version, credit card, swedish id number and swedish passport number are detected

In [178]:
import re
from termcolor import colored

def check_credit_card_no(candidate_value):

  candidate_value = re.sub('\D', '', str(candidate_value))
  x = re.search("^(?:4[0-9]{12}(?:[0-9]{3})?|[25][1-7][0-9]{14}|6(?:011|5[0-9][0-9])[0-9]{12}|3[47][0-9]{13}|3(?:0[0-5]|[68][0-9])[0-9]{11}|(?:2131|1800|35\d{3})\d{11})$", candidate_value)

  if x:
    return 1
  else:
    return 0


def check_phone_number():
  return


def check_date(candidate_value):
  x = re.search("^((19|2[0-9])[0-9]{2})-(0[1-9]|1[012])-(0[1-9]|[12][0-9]|3[01])$", str(candidate_value))

  if x:
    return 1
  else:
    return 0


def check_swedish_id(candidate_value):
  x = re.search("^\d{6,8}[-|(\s)]{0,1}\d{4}$", str(candidate_value))

  if x:
    return 1
  else:
    return 0


def check_swedish_passport_no(candidate_value):
  x = re.search("^[0-9]{8}$", str(candidate_value))

  if x:
    return 1
  else:
    return 0



---



The method that validates each column by the provided validation methods

In [179]:
def get_sensitive_data_insight(data_df, sensitive_threshold=0.8):

  red_color_code = '\u001b[31m' 

  column_name_list = list(data_df.columns)
  sensitive_data_list = []
  number_of_rows = data_df.shape[0]
  

  #validation functions names
  func_name_list = [check_swedish_passport_no, check_swedish_id, check_credit_card_no]

  for func_name in func_name_list:

    check_df = pd.DataFrame()

    for column_name in column_name_list:
      check_df[column_name] = data_df.apply(lambda row: func_name(row[column_name]), axis=1)

    title = func_name.__name__.replace('check_', '').replace('_', ' ') + ' - match ratio of columns'  
    print(title)
    print('-' * len(title))

    ratio_flag = False

    for column_name in column_name_list:
      number_of_matches = check_df[column_name].sum()
      match_ratio = number_of_matches/number_of_rows
      if number_of_matches > 0:

        out_str = column_name + ": " + str(match_ratio)  
        ratio_flag = True

        if match_ratio >= sensitive_threshold:
          print(colored(out_str, 'red'))
          sensitive_data_list.append({'column_name':column_name, 'sensitive_data': func_name.__name__.replace('check_', '').replace('_', ' '), 'match_ratio': match_ratio})
        else: 
          print(out_str)

    if not ratio_flag:
      print('No match detected')    

    print()

  return sensitive_data_list

In [180]:
dataset_01_df = pd.read_csv('data_10001.csv')
dataset_02_df = pd.read_csv('data_10002.csv')
dataset_03_df = pd.read_csv('data_10003.csv')

In [181]:
dataset_01_df

,column_01,column_02,column_03,column_04,column_05,column_06,column_07
0,0.928784,99040217,20101101-4555,98339063,6677-8899-3465-5366,7,1
1,0.928784,78499217,19630701-8990,63544063,4912-8899-3465-8499,2,1
2,0.928784,67589076,19930701-8277,67589063,4912-8899-3465-5366,4,1
3,0.363746,1983050127,19930701-8277,67589076,4912-5674-3465-1234,4,0
4,0.345689,198305010,19930701-8277,67589076,4912-5674-3465-1234,5,0
5,0.345684,67589076,19930701-8277,67589076,4912-5674-3465-1234,9,1
6,0.928784,67589076,19830501-7274,67589076,4912-5674-3465-1234,1,1
7,0.937584,67589076,19830501-7274,67587377,4912567434651234,8,0
8,0.743484,67589076,198305017274,67587377,4912567434651234,9,0
9,0.929484,19870217,19830501-7274,67587377,4912-5674-3465-1234,10,1


In [182]:
dataset_03_df

,c1,c2,c3
0,19830501-7274,67589076,1
1,19830501,67589076,1
2,67589076,67589076,1
3,19870217,67589076,2


In [183]:
dataset_03_df

,c1,c2,c3
0,19830501-7274,67589076,1
1,19830501,67589076,1
2,67589076,67589076,1
3,19870217,67589076,2




---



The sensitive data ratio of each column for dataset

In [184]:
sensitive_data_insight_01 = get_sensitive_data_insight(dataset_01_df)

swedish passport no - match ratio of columns
--------------------------------------------
column_02: 0.8333333333333334
column_04: 1.0

swedish id - match ratio of columns
-----------------------------------
column_02: 0.08333333333333333
column_03: 1.0

credit card no - match ratio of columns
---------------------------------------
column_05: 0.9166666666666666



In [185]:
sensitive_data_insight_02 = get_sensitive_data_insight(dataset_02_df)

swedish passport no - match ratio of columns
--------------------------------------------
column_b: 0.75
column_d: 1.0

swedish id - match ratio of columns
-----------------------------------
column_b: 0.25
column_c: 1.0

credit card no - match ratio of columns
---------------------------------------
column_e: 1.0



In [186]:
sensitive_data_insight_03 = get_sensitive_data_insight(dataset_03_df)

swedish passport no - match ratio of columns
--------------------------------------------
c1: 0.75
c2: 1.0

swedish id - match ratio of columns
-----------------------------------
c1: 0.25

credit card no - match ratio of columns
---------------------------------------
No match detected



In [187]:
sensitive_data_insight_01

[{'column_name': 'column_02',
  'match_ratio': 0.8333333333333334,
  'sensitive_data': 'swedish passport no'},
 {'column_name': 'column_04',
  'match_ratio': 1.0,
  'sensitive_data': 'swedish passport no'},
 {'column_name': 'column_03',
  'match_ratio': 1.0,
  'sensitive_data': 'swedish id'},
 {'column_name': 'column_05',
  'match_ratio': 0.9166666666666666,
  'sensitive_data': 'credit card no'}]

In [188]:
sensitive_data_insight_02

[{'column_name': 'column_d',
  'match_ratio': 1.0,
  'sensitive_data': 'swedish passport no'},
 {'column_name': 'column_c',
  'match_ratio': 1.0,
  'sensitive_data': 'swedish id'},
 {'column_name': 'column_e',
  'match_ratio': 1.0,
  'sensitive_data': 'credit card no'}]

In [189]:
sensitive_data_insight_03

[{'column_name': 'c2',
  'match_ratio': 1.0,
  'sensitive_data': 'swedish passport no'}]

In [190]:
def append_sensitive_data(data_insight_list, dataset_id):
  result_df = pd.json_normalize(data_insight_list)
  result_df['dataset_id'] = dataset_id

  return result_df

In [191]:
sentitive_data_01 = append_sensitive_data(sensitive_data_insight_01, 10001)
sentitive_data_02 = append_sensitive_data(sensitive_data_insight_02, 10002)
sentitive_data_03 = append_sensitive_data(sensitive_data_insight_03, 10003)

sentitive_data_df = pd.concat([sentitive_data_01, sentitive_data_02, sentitive_data_03])

sentitive_data_df.to_csv('sensitive_data_table.csv', index=False)

In [192]:
sensitive_data_df = pd.read_csv('sensitive_data_table.csv')
sensitive_data_df

,column_name,sensitive_data,match_ratio,dataset_id
0,column_02,swedish passport no,0.833333,10001
1,column_04,swedish passport no,1.000000,10001
2,column_03,swedish id,1.000000,10001
3,column_05,credit card no,0.916667,10001
4,column_d,swedish passport no,1.000000,10002
5,column_c,swedish id,1.000000,10002
6,column_e,credit card no,1.000000,10002
7,c2,swedish passport no,1.000000,10003




---



Retrieving data without sensitive columns with customer - dataset access validation

In [193]:
def get_sensitive_columns(dataset_id):
  return sensitive_data_df[sensitive_data_df['dataset_id'].astype(str) == str(dataset_id)]['column_name'].tolist()
  

def retrieve_data(customer_id, dataset_id):

  customer_dataset_df = pd.read_csv('customer_dataset_table.csv')

  customer_id_list = pd.read_csv('customer_table.csv')['customer_id'].tolist()
  dataset_id_list = pd.read_csv('dataset_table.csv')['dataset_id'].tolist()
  
  error_message = ""

  if customer_id not in customer_id_list:
    error_message = " Customer id " + str(customer_id) + " not found"

  if dataset_id not in dataset_id_list:
    error_message = error_message + "\n" + "Dataset id " + str(dataset_id) + " not found"

  if error_message != "":
    print(error_message[1:])
    return

  customer_dataset_list = customer_dataset_df[customer_dataset_df['customer_id'] == customer_id]['dataset_id'].tolist()

  if dataset_id not in customer_dataset_list:
    print('Customer ' + str(customer_id) + ' does not have access right to the dataset ' + str(dataset_id))
    return 

  data_df = pd.read_csv('data_' + str(dataset_id) + ".csv")

  sensitive_column_list = get_sensitive_columns(dataset_id)
  column_list = list(data_df.columns)
  out_column_list = list(set(column_list) - set(sensitive_column_list))
  out_column_list.sort()

  return data_df[out_column_list]



---



Sample usage

In [194]:
retrieve_data(101, 10002)

,column_a,column_b,column_f
0,0.743484,19830501-7274,a
1,0.929484,19830501,a
2,0.241239,67589076,b
3,0.929835,19870217,a


In [195]:
retrieve_data(101, 10001)

,column_01,column_06,column_07
0,0.928784,7,1
1,0.928784,2,1
2,0.928784,4,1
3,0.363746,4,0
4,0.345689,5,0
5,0.345684,9,1
6,0.928784,1,1
7,0.937584,8,0
8,0.743484,9,0
9,0.929484,10,1


In [196]:
retrieve_data(102, 10002)

,column_a,column_b,column_f
0,0.743484,19830501-7274,a
1,0.929484,19830501,a
2,0.241239,67589076,b
3,0.929835,19870217,a


In [197]:
retrieve_data(101, 10003)

Customer 101 does not have access right to the dataset 10003


In [198]:
retrieve_data(107, 10003)

Customer id 107 not found


In [199]:
retrieve_data(101, 10008)

Dataset id 10008 not found


In [200]:
retrieve_data(109, 10009)

Customer id 109 not found
Dataset id 10009 not found
